1등하신 분의 코드를 로컬에서 실행해본 것

In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import random
import os
import warnings

In [3]:
warnings.filterwarnings('ignore')

In [4]:
def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

SEED = 42

In [13]:
reset_seeds(SEED)

In [16]:
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import joblib

### Load Data

In [188]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
bi = pd.read_csv('building_info.csv')

In [189]:
date = pd.to_datetime(train_df['일시'])
train_df['hour'] = date.dt.hour
train_df['day'] = date.dt.weekday
train_df['month'] = date.dt.month
train_df['week'] = date.dt.weekofyear

In [190]:
date = pd.to_datetime(test_df['일시'])
test_df['hour'] = date.dt.hour
test_df['day'] = date.dt.weekday
test_df['month'] = date.dt.month
test_df['week'] = date.dt.weekofyear

building info의 태양광용량, ESS저장용량, PCS용량 열에서 '-'를 0으로 대체

In [191]:
cols = ['태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)']
for col in cols:
    bi[col] = bi[col].apply(lambda x: 0 if x=='-' else float(x))

### 결측치 처리

In [192]:
train_ft = train_df.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)'])
test_ft = test_df.drop(columns=['num_date_time', '일시'])

In [193]:
train_ft['강수량(mm)'] = train_ft['강수량(mm)'].fillna(0)

풍속과 습도의 경우 통계량으로 대체하는 것이 가장 성능이 좋았다고 함

그에 따라 해당 건물별 월, 시간을 기준으로 평균을 낸 값으로 대체

In [194]:
for i in train_ft[train_ft['풍속(m/s)'].isnull()].index:
    month = train_ft.iloc[i]['month']
    hour = train_ft.iloc[i]['hour']
    num = train_ft.iloc[i]['건물번호']
    data_mean = train_ft[(train_ft['건물번호']==num)&(train_ft['month']==month)&(train_ft['hour']==hour)]['풍속(m/s)'].describe()['mean']
    train_ft.loc[i, '풍속(m/s)'] = data_mean

In [195]:
for i in train_ft[train_ft['습도(%)'].isnull()].index:
    month = train_ft.iloc[i]['month']
    hour = train_ft.iloc[i]['hour']
    num = train_ft.iloc[i]['건물번호']
    data_mean = train_ft[(train_ft['건물번호']==num)&(train_ft['month']==month)&(train_ft['hour']==hour)]['습도(%)'].describe()['mean']
    train_ft.loc[i, '습도(%)'] = data_mean

### building info와 merge

In [196]:
train_ft = train_ft.merge(bi, how='left', on='건물번호')
test_ft = test_ft.merge(bi, how='left', on='건물번호')

### 범주형 인코딩

In [197]:
conv_dict=  dict(zip(list(train_ft['건물유형'].unique()), [0,1,2,3,4,5,6,7,8,9,10,11]))

In [198]:
train_ft['건물유형'] = train_ft['건물유형'].map(conv_dict)
test_ft['건물유형'] = test_ft['건물유형'].map(conv_dict)

### Target 생성

In [199]:
target = train_df['전력소비량(kWh)']

### 휴일 라벨링

In [200]:
train_ft['holiday'] = train_ft.apply(lambda x: 0 if x['day']<5 else 1, axis=1)
test_ft['holiday'] = train_ft.apply(lambda x: 0 if x['day']<5 else 1, axis=1)

### Cyclical Encoding

In [201]:
train_ft['sin_time'] = np.sin(2*np.pi*train_ft.hour/24)
train_ft['cos_time'] = np.cos(2*np.pi*train_ft.hour/24)
test_ft['sin_time'] = np.sin(2*np.pi*test_ft.hour/24)
test_ft['cos_time'] = np.cos(2*np.pi*test_ft.hour/24)

### 불쾌지수

In [202]:
train_ft['THI'] = 1.8*train_ft['기온(C)'] - 0.55*(1-train_ft['습도(%)']/100)*(1.8*train_ft['기온(C)']-26)+32
test_ft['THI'] = 1.8*test_ft['기온(C)'] - 0.55*(1-test_ft['습도(%)']/100)*(1.8*test_ft['기온(C)']-26)+32

단순히 수치형으로 사용하는 것보다 범주화를 진행했을 때 높은 성능을 보여 범주형 feature로 사용

In [203]:
train_ft['THI'] = pd.cut(train_ft['THI'], bins=[0,68,75,80,200], labels=[1,2,3,4])
train_ft['THI'] = train_ft['THI'].astype(int)
test_ft['THI'] = pd.cut(test_ft['THI'], bins=[0,68,75,80,200], labels=[1,2,3,4])
test_ft['THI'] = test_ft['THI'].astype(int)

### 이전 10일 간의 (기온 - 권장온도)의 합

In [204]:
def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

In [205]:
cdhs = np.array([])
for num in range(1, 101, 1):
    temp = train_ft[train_ft['건물번호']==num]
    cdh = CDH(temp['기온(C)'].values)
    cdhs = np.concatenate([cdhs, cdh])
train_ft['CDH'] = cdhs

In [206]:
cdhs = np.array([])
for num in range(1, 101, 1):
    temp = test_ft[test_ft['건물번호']==num]
    cdh = CDH(temp['기온(C)'].values)
    cdhs = np.concatenate([cdhs, cdh])
test_ft['CDH'] = cdhs

### 파생 변수들
- target의 다양한 통계치를 feature로 추가

### 요일 시간별 평균 사용량

In [207]:
power_mean = pd.pivot_table(train_ft, values = '전력소비량(kWh)', index = ['건물번호', 'hour', 'day','month'], aggfunc = np.mean).reset_index()

power_mean = power_mean.drop_duplicates(subset=['건물번호', 'hour', 'day'], keep='first')

train_ft = train_ft.merge(power_mean[['건물번호', 'hour', 'day', '전력소비량(kWh)']],
                          on=['건물번호', 'hour', 'day'],
                          how='left',
                          suffixes=('', '_mean'))
train_ft.rename(columns={'전력소비량(kWh)_mean': 'day_hour_mean'}, inplace=True)

test_ft = test_ft.merge(power_mean[['건물번호', 'hour', 'day', '전력소비량(kWh)']],
                        on=['건물번호', 'hour', 'day'],
                        how='left',
                        suffixes=('', '_mean'))
test_ft.rename(columns={'전력소비량(kWh)': 'day_hour_mean'}, inplace=True)

### 월 시간별 평균 사용량

In [208]:
power_mean = pd.pivot_table(train_ft, values = '전력소비량(kWh)', index = ['건물번호', 'hour', 'day','month'], aggfunc = np.mean).reset_index()

power_mean = power_mean.drop_duplicates(subset=['건물번호', 'hour', 'month'], keep='first')

train_ft = train_ft.merge(power_mean[['건물번호', 'hour', 'month', '전력소비량(kWh)']],
                          on=['건물번호', 'hour', 'month'],
                          how='left',
                          suffixes=('', '_mean'))
train_ft.rename(columns={'전력소비량(kWh)_mean': 'month_hour_mean'}, inplace=True)

test_ft = test_ft.merge(power_mean[['건물번호', 'hour', 'month', '전력소비량(kWh)']],
                        on=['건물번호', 'hour', 'month'],
                        how='left',
                        suffixes=('', '_mean'))
test_ft.rename(columns={'전력소비량(kWh)': 'month_hour_mean'}, inplace=True)

### 시간별 평균 사용량

In [210]:
power_mean = pd.pivot_table(train_ft, values = '전력소비량(kWh)', index = ['건물번호', 'hour'], aggfunc = np.mean).reset_index()
train_ft = train_ft.merge(power_mean[['건물번호', 'hour', '전력소비량(kWh)']],
                          on=['건물번호', 'hour'],
                          how='left',
                          suffixes=('', '_mean'))
train_ft.rename(columns={'전력소비량(kWh)_mean': 'hour_mean'}, inplace=True)

test_ft = test_ft.merge(power_mean[['건물번호', 'hour', '전력소비량(kWh)']],
                        on=['건물번호', 'hour'],
                        how='left',
                        suffixes=('', '_mean'))
test_ft.rename(columns={'전력소비량(kWh)': 'hour_mean'}, inplace=True)

### 시간별 사용량 표준편차

In [212]:
power_std = pd.pivot_table(train_ft, values = '전력소비량(kWh)', index = ['건물번호', 'hour'], aggfunc = np.std).reset_index()
train_ft = train_ft.merge(power_std[['건물번호', 'hour', '전력소비량(kWh)']],
                          on=['건물번호', 'hour'],
                          how='left',
                          suffixes=('', '_std'))
train_ft.rename(columns={'전력소비량(kWh)_std': 'hour_std'}, inplace=True)

test_ft = test_ft.merge(power_std[['건물번호', 'hour', '전력소비량(kWh)']],
                        on=['건물번호', 'hour'],
                        how='left',
                        suffixes=('', '_std'))
test_ft.rename(columns={'전력소비량(kWh)': 'hour_std'}, inplace=True)

### 요일별 시간 사용량 표준편차

In [213]:
power_std = pd.pivot_table(train_ft, values = '전력소비량(kWh)', index = ['건물번호', 'hour', 'day'], aggfunc = np.std).reset_index()
train_ft = train_ft.merge(power_std[['건물번호', 'hour', 'day', '전력소비량(kWh)']],
                          on=['건물번호', 'hour', 'day'],
                          how='left',
                          suffixes=('', '_std'))
train_ft.rename(columns={'전력소비량(kWh)_std': 'day_hour_std'}, inplace=True)

test_ft = test_ft.merge(power_std[['건물번호', 'hour', 'day', '전력소비량(kWh)']],
                        on=['건물번호', 'hour', 'day'],
                        how='left',
                        suffixes=('', '_std'))
test_ft.rename(columns={'전력소비량(kWh)': 'day_hour_std'}, inplace=True)

### 요일별 시간 사용량 중앙값

In [214]:
power_median = pd.pivot_table(train_ft, values = '전력소비량(kWh)', index = ['건물번호', 'hour', 'day'], aggfunc = np.median).reset_index()
train_ft = train_ft.merge(power_median[['건물번호', 'hour', 'day', '전력소비량(kWh)']],
                          on=['건물번호', 'hour', 'day'],
                          how='left',
                          suffixes=('', '_median'))
train_ft.rename(columns={'전력소비량(kWh)_median': 'day_hour_median'}, inplace=True)

test_ft = test_ft.merge(power_median[['건물번호', 'hour', 'day', '전력소비량(kWh)']],
                        on=['건물번호', 'hour', 'day'],
                        how='left',
                        suffixes=('', '_median'))
test_ft.rename(columns={'전력소비량(kWh)': 'day_hour_median'}, inplace=True)

In [215]:
train_ft = train_ft.drop(columns=['holiday','건물유형','강수량(mm)','풍속(m/s)','전력소비량(kWh)'])
test_ft = test_ft.drop(columns=['holiday','건물유형','강수량(mm)','풍속(m/s)'])

In [216]:
cols = ['건물번호', '기온(C)', '습도(%)', 'hour', 'day', 'month', 'week', '연면적(m2)',
       '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)', 'sin_time',
       'cos_time', 'THI', 'CDH', 'day_hour_mean', 'month_hour_mean','hour_mean', 'hour_std','day_hour_std',
       'day_hour_median']

In [217]:
test_ft.columns

Index(['건물번호', '기온(C)', '습도(%)', 'hour', 'day', 'month', 'week', '연면적(m2)',
       '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)', 'sin_time',
       'cos_time', 'THI', 'CDH', 'day_hour_mean', 'month_hour_mean',
       'hour_mean', 'hour_std', 'day_hour_std', 'day_hour_median'],
      dtype='object')

In [218]:
train_ft = train_ft[cols]
test_ft = test_ft[cols]

In [221]:
train_ft['전력소비량(kWh)'] = train_df['전력소비량(kWh)']

### 클러스터링

In [222]:
train_ft['holiday'] = train_ft.apply(lambda x: 0 if x['day']<5 else 1, axis=1)
test_ft['holiday'] = test_ft.apply(lambda x: 0 if x['day']<5 else 1, axis=1)

In [223]:
weekday_mean = train_ft[train_ft.holiday==0].pivot_table(values='전력소비량(kWh)', index=train_ft.건물번호, columns='hour', aggfunc='first')
weekend_mean = train_ft[train_ft.holiday==1].pivot_table(values='전력소비량(kWh)', index=train_ft.건물번호, columns='hour', aggfunc='first')

In [224]:
tmp = pd.merge(weekday_mean, weekend_mean, how='left', on='건물번호')
tmp

hour,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,14_y,15_y,16_y,17_y,18_y,19_y,20_y,21_y,22_y,23_y
건물번호,,,,,,,,,,,,,,,,,,,,,
1,1085.28,1047.36,974.88,953.76,986.40,1087.20,1314.72,1684.80,1976.16,2289.12,...,3210.24,3153.60,3035.52,2726.40,2315.04,2229.60,2069.76,1733.76,1414.08,1191.84
2,1170.36,1146.96,1115.28,1107.00,1069.92,1075.32,1116.36,1254.24,887.40,1035.00,...,2245.68,2316.60,2528.28,2201.04,2028.60,1636.56,1380.96,1109.52,811.44,1268.64
3,926.28,884.52,882.36,867.24,880.92,808.56,786.24,809.64,818.64,882.36,...,1596.96,1791.72,1875.60,1746.00,1794.24,2265.48,2087.28,1052.64,972.72,912.60
4,740.16,671.04,619.20,626.40,626.22,630.36,752.04,829.44,957.60,1093.14,...,1237.50,1226.52,1172.70,1064.88,959.04,875.88,816.12,755.10,708.48,656.28
5,1304.64,1308.24,1302.48,1295.28,1316.88,1398.96,1492.56,1535.04,1548.72,1560.96,...,5785.92,5768.64,5553.36,5284.80,2644.56,1645.92,1538.64,1463.04,1404.00,1416.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1576.68,1558.68,1413.36,1483.92,1438.56,1503.84,1738.80,2073.36,2263.80,2621.04,...,3095.16,3043.80,3042.36,2958.84,2883.48,2780.04,2729.28,2525.88,2098.44,1848.72
97,774.36,553.86,558.90,559.44,559.62,582.48,640.26,903.78,855.18,907.56,...,1486.44,1469.88,1471.50,1340.46,1215.90,1180.98,1275.30,1128.42,975.96,895.14
98,533.52,516.78,509.40,479.70,541.98,712.08,947.88,1004.58,1069.02,1133.64,...,1298.16,1296.72,1273.32,1298.70,1238.22,1139.58,1093.68,979.20,783.00,654.30


In [225]:
kmeans = KMeans(n_clusters=5, random_state=SEED, n_init='auto')
kmeans.fit(tmp)

KMeans(n_clusters=5, n_init='auto', random_state=42)

In [226]:
cluster_ = kmeans.predict(tmp)

In [227]:
tmp['cluster'] = cluster_

In [228]:
train_ft = train_ft.merge(tmp.cluster, how='left', on='건물번호')
test_ft = test_ft.merge(tmp.cluster, how='left', on='건물번호')

clustering 결과와 holiday를 같이 사용하면 성능이 하락하여 holiday 제거

In [229]:
train_ft = train_ft.drop(columns=['holiday'])
test_ft = test_ft.drop(columns=['holiday'])

In [230]:
power_mean = pd.pivot_table(train_ft, values = '전력소비량(kWh)', index = ['cluster', 'hour', 'day','month'], aggfunc = np.mean).reset_index()

power_mean = power_mean.drop_duplicates(subset=['cluster', 'hour', 'day'], keep='first')

train_ft = train_ft.merge(power_mean[['cluster', 'hour', 'day', '전력소비량(kWh)']],
                          on=['cluster', 'hour', 'day'],
                          how='left',
                          suffixes=('', '_mean'))
train_ft.rename(columns={'전력소비량(kWh)_mean': 'cluster_day_hour_mean'}, inplace=True)

test_ft = test_ft.merge(power_mean[['cluster', 'hour', 'day', '전력소비량(kWh)']],
                        on=['cluster', 'hour', 'day'],
                        how='left',
                        suffixes=('', '_mean'))
test_ft.rename(columns={'전력소비량(kWh)': 'cluster_day_hour_mean'}, inplace=True)

In [231]:
display(train_ft)
display(test_ft)

,건물번호,기온(C),습도(%),hour,day,month,week,연면적(m2),냉방면적(m2),태양광용량(kW),...,CDH,day_hour_mean,month_hour_mean,hour_mean,hour_std,day_hour_std,day_hour_median,전력소비량(kWh),cluster,cluster_day_hour_mean
0,1,18.6,42.0,0,2,6,22,110634.00,39570.00,0.0,...,-7.4,1174.656,1189.44,1706.318118,446.882767,517.982222,2094.72,1085.28,0,2005.185000
1,1,18.0,45.0,1,2,6,22,110634.00,39570.00,0.0,...,-15.4,1101.600,1131.12,1622.620235,439.662704,500.769931,2015.52,1047.36,0,1952.292000
2,1,17.7,45.0,2,2,6,22,110634.00,39570.00,0.0,...,-23.7,1025.760,1063.56,1506.971294,412.071906,465.227458,1848.00,974.88,0,1918.248500
3,1,16.7,48.0,3,2,6,22,110634.00,39570.00,0.0,...,-33.0,1015.584,1042.68,1437.365647,391.205981,436.601091,1764.00,953.76,0,1894.579500
4,1,18.4,43.0,4,2,6,22,110634.00,39570.00,0.0,...,-40.6,1041.792,1075.44,1447.321412,381.099697,405.518091,1619.52,986.40,0,1901.786500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,23.1,86.0,19,2,8,34,57497.84,40035.23,0.0,...,-19.6,832.752,1168.20,1010.462118,161.399578,163.825489,1017.12,881.04,4,1424.581127
203996,100,22.4,86.0,20,2,8,34,57497.84,40035.23,0.0,...,-20.2,773.040,1026.00,928.125176,137.566008,153.076049,874.32,798.96,4,1353.478945
203997,100,21.3,92.0,21,2,8,34,57497.84,40035.23,0.0,...,-22.3,686.784,934.68,830.032941,128.300189,143.415686,781.20,825.12,4,1228.707309
203998,100,21.0,94.0,22,2,8,34,57497.84,40035.23,0.0,...,-25.1,584.736,815.52,723.100235,112.464079,105.147190,657.12,640.08,4,1073.485988


,건물번호,기온(C),습도(%),hour,day,month,week,연면적(m2),냉방면적(m2),태양광용량(kW),...,THI,CDH,day_hour_mean,month_hour_mean,hour_mean,hour_std,day_hour_std,day_hour_median,cluster,cluster_day_hour_mean
0,1,23.5,72,0,3,8,34,110634.00,39570.00,0.0,...,2,-2.5,1135.584,2237.04,1706.318118,446.882767,446.984913,1847.28,0,2011.061500
1,1,23.0,72,1,3,8,34,110634.00,39570.00,0.0,...,2,-5.5,1054.848,2165.64,1622.620235,439.662704,449.091398,1772.40,0,1972.697000
2,1,22.7,75,2,3,8,34,110634.00,39570.00,0.0,...,2,-8.8,975.648,2035.92,1506.971294,412.071906,415.453568,1624.80,0,1943.292000
3,1,22.1,78,3,3,8,34,110634.00,39570.00,0.0,...,2,-12.7,956.736,1954.56,1437.365647,391.205981,378.117772,1557.84,0,1925.145000
4,1,21.8,77,4,3,8,34,110634.00,39570.00,0.0,...,2,-16.9,984.768,1960.56,1447.321412,381.099697,360.348483,1561.68,0,1936.258500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,22.5,84,19,2,8,35,57497.84,40035.23,0.0,...,2,-34.5,832.752,1168.20,1010.462118,161.399578,163.825489,1017.12,4,1424.581127
16796,100,20.7,95,20,2,8,35,57497.84,40035.23,0.0,...,2,-34.4,773.040,1026.00,928.125176,137.566008,153.076049,874.32,4,1353.478945
16797,100,20.2,98,21,2,8,35,57497.84,40035.23,0.0,...,2,-35.3,686.784,934.68,830.032941,128.300189,143.415686,781.20,4,1228.707309
16798,100,20.1,97,22,2,8,35,57497.84,40035.23,0.0,...,2,-36.8,584.736,815.52,723.100235,112.464079,105.147190,657.12,4,1073.485988


In [232]:
train_ft = train_ft.drop(columns=['전력소비량(kWh)'])

### 이동평균

In [233]:
window_size = 96

train_ft['기온_4일_이동평균'] = train_ft.groupby('건물번호')['기온(C)'].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())
train_ft['습도_4일_이동평균'] = train_ft.groupby('건물번호')['습도(%)'].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())

window_size = 168

train_ft['기온_7일_이동평균'] = train_ft.groupby('건물번호')['기온(C)'].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())
train_ft['습도_7일_이동평균'] = train_ft.groupby('건물번호')['습도(%)'].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())

In [234]:
train_ft.head()

,건물번호,기온(C),습도(%),hour,day,month,week,연면적(m2),냉방면적(m2),태양광용량(kW),...,hour_mean,hour_std,day_hour_std,day_hour_median,cluster,cluster_day_hour_mean,기온_4일_이동평균,습도_4일_이동평균,기온_7일_이동평균,습도_7일_이동평균
0,1,18.6,42.0,0,2,6,22,110634.0,39570.0,0.0,...,1706.318118,446.882767,517.982222,2094.72,0,2005.1850,18.60,42.0,18.60,42.0
1,1,18.0,45.0,1,2,6,22,110634.0,39570.0,0.0,...,1622.620235,439.662704,500.769931,2015.52,0,1952.2920,18.30,43.5,18.30,43.5
2,1,17.7,45.0,2,2,6,22,110634.0,39570.0,0.0,...,1506.971294,412.071906,465.227458,1848.00,0,1918.2485,18.10,44.0,18.10,44.0
3,1,16.7,48.0,3,2,6,22,110634.0,39570.0,0.0,...,1437.365647,391.205981,436.601091,1764.00,0,1894.5795,17.75,45.0,17.75,45.0
4,1,18.4,43.0,4,2,6,22,110634.0,39570.0,0.0,...,1447.321412,381.099697,405.518091,1619.52,0,1901.7865,17.88,44.6,17.88,44.6


In [235]:
for i in range(1,101):
  test_ft = pd.concat([train_ft[train_ft['건물번호']==i],test_ft],axis=0)

In [236]:
window_size = 96

test_ft['기온_4일_이동평균'] = test_ft.groupby('건물번호')['기온(C)'].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())
test_ft['습도_4일_이동평균'] = test_ft.groupby('건물번호')['습도(%)'].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())

window_size = 168

test_ft['기온_7일_이동평균'] = test_ft.groupby('건물번호')['기온(C)'].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())
test_ft['습도_7일_이동평균'] = test_ft.groupby('건물번호')['습도(%)'].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())

test_ft.head()

,건물번호,기온(C),습도(%),hour,day,month,week,연면적(m2),냉방면적(m2),태양광용량(kW),...,hour_mean,hour_std,day_hour_std,day_hour_median,cluster,cluster_day_hour_mean,기온_4일_이동평균,습도_4일_이동평균,기온_7일_이동평균,습도_7일_이동평균
201960,100,15.0,70.0,0,2,6,22,57497.84,40035.23,0.0,...,450.982588,57.618897,67.693627,430.80,4,899.816000,15.000000,70.00,15.000000,70.00
201961,100,14.2,75.0,1,2,6,22,57497.84,40035.23,0.0,...,404.450824,48.260651,60.788147,387.36,4,867.600073,14.600000,72.50,14.600000,72.50
201962,100,13.5,77.0,2,2,6,22,57497.84,40035.23,0.0,...,381.910588,46.060484,59.132642,365.52,4,842.552400,14.233333,74.00,14.233333,74.00
201963,100,13.4,81.0,3,2,6,22,57497.84,40035.23,0.0,...,372.900706,44.517603,57.770095,362.16,4,835.787570,14.025000,75.75,14.025000,75.75
201964,100,13.8,79.0,4,2,6,22,57497.84,40035.23,0.0,...,369.554824,42.833213,54.814466,352.80,4,835.758655,13.980000,76.40,13.980000,76.40


In [237]:
test_ft = test_ft.iloc[-16800:]

In [238]:
display(train_ft)
display(test_ft)

,건물번호,기온(C),습도(%),hour,day,month,week,연면적(m2),냉방면적(m2),태양광용량(kW),...,hour_mean,hour_std,day_hour_std,day_hour_median,cluster,cluster_day_hour_mean,기온_4일_이동평균,습도_4일_이동평균,기온_7일_이동평균,습도_7일_이동평균
0,1,18.6,42.0,0,2,6,22,110634.00,39570.00,0.0,...,1706.318118,446.882767,517.982222,2094.72,0,2005.185000,18.600000,42.000000,18.600000,42.000000
1,1,18.0,45.0,1,2,6,22,110634.00,39570.00,0.0,...,1622.620235,439.662704,500.769931,2015.52,0,1952.292000,18.300000,43.500000,18.300000,43.500000
2,1,17.7,45.0,2,2,6,22,110634.00,39570.00,0.0,...,1506.971294,412.071906,465.227458,1848.00,0,1918.248500,18.100000,44.000000,18.100000,44.000000
3,1,16.7,48.0,3,2,6,22,110634.00,39570.00,0.0,...,1437.365647,391.205981,436.601091,1764.00,0,1894.579500,17.750000,45.000000,17.750000,45.000000
4,1,18.4,43.0,4,2,6,22,110634.00,39570.00,0.0,...,1447.321412,381.099697,405.518091,1619.52,0,1901.786500,17.880000,44.600000,17.880000,44.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,23.1,86.0,19,2,8,34,57497.84,40035.23,0.0,...,1010.462118,161.399578,163.825489,1017.12,4,1424.581127,25.161458,87.552083,25.341667,86.809524
203996,100,22.4,86.0,20,2,8,34,57497.84,40035.23,0.0,...,928.125176,137.566008,153.076049,874.32,4,1353.478945,25.127083,87.531250,25.327976,86.875000
203997,100,21.3,92.0,21,2,8,34,57497.84,40035.23,0.0,...,830.032941,128.300189,143.415686,781.20,4,1228.707309,25.094792,87.489583,25.321429,86.892857
203998,100,21.0,94.0,22,2,8,34,57497.84,40035.23,0.0,...,723.100235,112.464079,105.147190,657.12,4,1073.485988,25.063542,87.458333,25.311905,86.928571


,건물번호,기온(C),습도(%),hour,day,month,week,연면적(m2),냉방면적(m2),태양광용량(kW),...,hour_mean,hour_std,day_hour_std,day_hour_median,cluster,cluster_day_hour_mean,기온_4일_이동평균,습도_4일_이동평균,기온_7일_이동평균,습도_7일_이동평균
0,1,23.5,72.0,0,3,8,34,110634.00,39570.00,0.0,...,1706.318118,446.882767,446.984913,1847.28,0,2011.061500,26.292708,73.385417,26.369643,76.553571
1,1,23.0,72.0,1,3,8,34,110634.00,39570.00,0.0,...,1622.620235,439.662704,449.091398,1772.40,0,1972.697000,26.253125,73.270833,26.367262,76.494048
2,1,22.7,75.0,2,3,8,34,110634.00,39570.00,0.0,...,1506.971294,412.071906,415.453568,1624.80,0,1943.292000,26.211458,73.166667,26.364881,76.440476
3,1,22.1,78.0,3,3,8,34,110634.00,39570.00,0.0,...,1437.365647,391.205981,378.117772,1557.84,0,1925.145000,26.167708,73.093750,26.358333,76.404762
4,1,21.8,77.0,4,3,8,34,110634.00,39570.00,0.0,...,1447.321412,381.099697,360.348483,1561.68,0,1936.258500,26.129167,73.041667,26.352976,76.345238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,22.5,84.0,19,2,8,35,57497.84,40035.23,0.0,...,1010.462118,161.399578,163.825489,1017.12,4,1424.581127,20.070833,86.343750,20.854762,83.607143
16796,100,20.7,95.0,20,2,8,35,57497.84,40035.23,0.0,...,928.125176,137.566008,153.076049,874.32,4,1353.478945,20.080208,86.593750,20.844643,83.660714
16797,100,20.2,98.0,21,2,8,35,57497.84,40035.23,0.0,...,830.032941,128.300189,143.415686,781.20,4,1228.707309,20.108333,86.760417,20.838095,83.696429
16798,100,20.1,97.0,22,2,8,35,57497.84,40035.23,0.0,...,723.100235,112.464079,105.147190,657.12,4,1073.485988,20.141667,86.875000,20.832738,83.714286


In [239]:
cv = KFold(n_splits = 5, shuffle=True, random_state=SEED)

In [240]:
model3 = XGBRegressor(random_state=SEED)

In [241]:
model2 = LGBMRegressor(random_state=SEED)

In [242]:
def SMAPE(true, pred):
    return np.mean((np.abs(true-pred)*2)/(np.abs(true) + np.abs(pred))) * 100

In [243]:
train_ft2 = train_ft.copy()
test_ft2 = test_ft.copy()

In [245]:
train_ft = train_ft2[train_ft2.columns[:-2]]
test_ft = test_ft2[test_ft2.columns[:-2]]

In [246]:
scores = cross_val_score(model3, train_ft, target, cv = cv, scoring=make_scorer(SMAPE, greater_is_better=False), n_jobs=-1)
print(f'XGB: {-np.mean(scores)}')

XGB: 4.996570584081963


### 추가 Feature Engineering

전반적으로 feature engineering 과정에서 일반화 성능의 향상을 위해 Feature Importance 및 SHAP 라이브러리를 통해서 각각의 feautre의 영향을 확인하며 특정 feature에 대해서 과도하게 의존하는 것을 줄이기 위해 노력함

### 학습

- Catboost, LGBM, XGBoost 총 3가지 모델을 활용하여 Ensemble
- 학습과정에서 일부 모델에 대해서 Categorical Feature를 바꿔가며 일반화 성능을 향상시키고자 함
- 5-Fold
- seed ensemble

In [247]:
cat_col = ['건물번호','cluster','THI','day','week']
def lgbm_smape(preds, train_data):
    labels = train_data
    smape_val = SMAPE(labels, preds)
    return 'SMAPE', smape_val, False

In [249]:
reset_seeds(SEED)
models = []
for i in tqdm(range(42, 53)):
    print(f'========== [{i}] SEED ==========')
    cat_params = {
        'learning_rate': 0.14059048492476106,
        'loss_function': 'RMSE',
        'random_seed': i,
        'verbose': False,
        'train_dir': 'catboost_1690862972.3436863',
        'n_estimators': 10000,
        'early_stopping_rounds': 100,
        'objective': 'MAE',
     }

    lgbm_params = {
        'random_state': i,
        'n_estimators': 15000,
        'num_leaves': 8,
        'min_child_samples': 12,
        'learning_rate': 0.17010396907527026,
        'colsample_bytree': 0.9605563464803123,
        'reg_alpha': 0.1110993344544235,
        'reg_lambda': 0.7948637803974561,
        'verbose': -1
    }

    xgb_params = {
        'random_state': i,
        'n_estimators': 4682,
        'max_leaves': 101,
        'min_child_weight': 7.581207558322951,
        'learning_rate': 0.08979034933474227,
        'subsample': 0.8905280196300354,
        'colsample_bylevel': 1.0,
        'colsample_bytree': 0.9523645407001878,
        'reg_alpha': 0.006919296411231538,
        'reg_lambda': 0.0998936254543762
    }

    CAT_model = CatBoostRegressor(**cat_params)
    LGBM_model = LGBMRegressor(**lgbm_params)
    XGB_model = XGBRegressor(**xgb_params)

    scores = []
    for tri, vai in tqdm(cv.split(train_ft), total=5):
        x_train = train_ft.iloc[tri]
        y_train = target.iloc[tri]

        x_valid = train_ft.iloc[vai]
        y_valid = target.iloc[vai]

        XGB_model.fit(x_train, y_train)
        LGBM_model.fit(x_train, y_train, eval_metric=lgbm_smape, 
                       categorical_feature=cat_col,
                       eval_set=(x_valid, y_valid))
        CAT_model.fit(x_train, y_train)

        pred = XGB_model.predict(x_valid)
        pred2 = LGBM_model.predict(x_valid)
        pred3 = CAT_model.predict(x_valid)
        pred = (pred + pred2 + pred3) / 3

        score = SMAPE(y_valid, pred)
        print(score)
        scores.append(score)
        models.append(XGB_model)
        models.append(LGBM_model)
        models.append(CAT_model)
    print(np.mean(scores))    

  0%|          | 0/11 [00:00<?, ?it/s]

========== [42] SEED ==========


  0%|          | 0/5 [00:00<?, ?it/s]

3.2949788515754226
3.3313059304522796
3.320938557680465
3.27984433235241
3.27129211375013
3.299671957162141
========== [43] SEED ==========


  0%|          | 0/5 [00:00<?, ?it/s]

...

3.3066865602372797
3.3428653698622224
3.3470802582866095
3.2707720098906616
3.2653210124448293
3.3065450421443203
========== [51] SEED ==========


  0%|          | 0/5 [00:00<?, ?it/s]

3.301601070803127
3.32364992181847
3.3185824290205828
3.2723248697995353
3.266777465925555
3.296587151473454
========== [52] SEED ==========


  0%|          | 0/5 [00:00<?, ?it/s]

3.297066462378301
3.3242751176371246
3.33174752024194
3.2759756514067844
3.258507907990655
3.297514531930961


In [252]:
for i, model in enumerate(models):
    joblib.dump(model, f'first_price_test/eleven_seeds_ensemble[{i}].pkl')

1등은 여기서 추가로 기존 결과물들을 활용하여 가중 평균을 수행하여 최종 제출물을 생성함